# Bonus Question - Text generation (adopted from slides)

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import os
import time
import keras
from tensorflow.keras.layers import LSTM

### Download the text

In [3]:
df = pd.read_csv('https://zenodo.org/record/2587904/files/Greek_Parliament_Proceedings_1989_2019.csv.zip?download=1',compression='zip') 

In [4]:
df = df.dropna(subset=['political_party','speech'])[0:30000]

In [5]:
df.head()

,member_name,sitting_date,parliamentary_period,parliamentary_session,parliamentary_sitting,political_party,speaker_info,speech
0,κρητικος νικολαου παναγιωτης,1989-07-03,period 5,session 1,sitting 1,πανελληνιο σοσιαλιστικο κινημα,προεδρευων,Παρακαλείται ο Γραμματέας κ. Βουλγαράκης να σ...
1,κρητικος νικολαου παναγιωτης,1989-07-03,period 5,session 1,sitting 1,πανελληνιο σοσιαλιστικο κινημα,προεδρευων,Παρακαλείται ο κύριος Γραμματέας να συνοδεύσε...
2,κρητικος νικολαου παναγιωτης,1989-07-03,period 5,session 1,sitting 1,πανελληνιο σοσιαλιστικο κινημα,προεδρευων,"Κύριοι συνάδελφοι, παρακαλώ τη Βουλή να εξουσ..."
3,NaN,1989-07-03,period 5,session 1,sitting 1,βουλη,βουλευτης/ες,"Μάλιστα, μάλιστα."
4,κρητικος νικολαου παναγιωτης,1989-07-03,period 5,session 1,sitting 1,πανελληνιο σοσιαλιστικο κινημα,προεδρευων,Η Βουλή παρέσχε τη ζητηθείσα εξουσιοδότηση. Μ...


In [6]:
others = df['political_party']
speeches = df['speech']

In [7]:
parltext = speeches.str.cat(sep= '\n')

In [8]:
type(parltext[700:1000])

str

In [9]:
parltext[:3000]

' Παρακαλείται ο Γραμματέας κ. Βουλγαράκης να συνοδεύσει το Μακαριότατο Αρχιεπίσκοπο Αθηνών και πάσης Ελλάδος κ. ΣΕΡΑΦΕΙΜ και τα συνοδεύοντα αυτόν μέλη της Ιεράς Συνόδου κατά την είσοδό τους στην Αίθουσα της Βουλής, προκειμένου να τελεσθεί αγιασμός.  . Στη συνέχεια τελείται ο καθιερωμένος αγιασμός.\n Παρακαλείται ο κύριος Γραμματέας να συνοδεύσει την Ιερά Σύνοδο εκτός της Αιθούσης της Βουλής.  . Παρακαλείται ο συνάδελφος Βουλευτής κ. Σαδίκ Αμέτ, που ανήκει στο Μωαμεθανικό Θρήσκευμα να προσέλθει και να δώσει τον οριζόμενο από το Σύνταγμα όρκο επί του Κορανίου.  : ~"Ορκίζομαι στο όνομα του Παντοδύναμου Θεού και του μόνου αυτού Προφήτη ο οποίος είναι ο Μωάμεθ να είμαι πιστός στην πατρίδα και το δημοκρατικό πολίτευμα, να υπακούω στο Σύνταγμα και τους νόμους και να εκπληρώνω ευσυνείδητα τα καθήκοντά μου".\n Κύριοι συνάδελφοι, παρακαλώ τη Βουλή να εξουσιοδοτήσει το Προεδρείο για την επικύρωση των Πρακτικών της σημερινής συνεδριάσεως.\n Μάλιστα, μάλιστα.\n Η Βουλή παρέσχε τη ζητηθείσα εξουσιο

In [10]:
parlvocab = sorted(set(parltext))
print('{} unique characters'.format(len(parlvocab)))

163 unique characters


In [11]:
parlvocab

['\n',
 ' ',
 '!',
 '"',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~',
 '«',
 '»',
 '΄',
 'Ά',
 'Έ',
 'Ή',
 'Ί',
 'Ό',
 'Ύ',
 'Ώ',
 'ΐ',
 'Α',
 'Β',
 'Γ',
 'Δ',
 'Ε',
 'Ζ',
 'Η',
 'Θ',
 'Ι',
 'Κ',
 'Λ',
 'Μ',
 'Ν',
 'Ξ',
 'Ο',
 'Π',
 'Ρ',
 'Σ',
 'Τ',
 'Υ',
 'Φ',
 'Χ',
 'Ψ',
 'Ω',
 'Ϊ',
 'Ϋ',
 'ά',
 'έ',
 'ή',
 'ί',
 'ΰ',
 'α',
 'β',
 'γ',
 'δ',
 'ε',
 'ζ',
 'η',
 'θ',
 'ι',
 'κ',
 'λ',
 'μ',
 'ν',
 'ξ',
 'ο',
 'π',
 'ρ',
 'ς',
 'σ',
 'τ',
 'υ',
 'φ',
 'χ',
 'ψ',
 'ω',
 'ϊ',
 'ϋ',
 'ό',
 'ύ',
 'ώ',
 '–',
 '‘',
 '’',
 '…']

In [12]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(parlvocab))

In [13]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [14]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(parltext, 'UTF-8'))
all_ids

<tf.Tensor: shape=(26156086,), dtype=int64, numpy=array([  3, 115, 131, ..., 142, 135,  15])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('utf-8'))

 
Π
α
ρ
α
κ
α
λ
ε
ί


In [18]:
seq_length = 200
examples_per_epoch = len(parltext)//(seq_length+1)

In [19]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b' ' b'\xce\xa0' b'\xce\xb1' b'\xcf\x81' b'\xce\xb1' b'\xce\xba'
 b'\xce\xb1' b'\xce\xbb' b'\xce\xb5' b'\xce\xaf' b'\xcf\x84' b'\xce\xb1'
 b'\xce\xb9' b' ' b'\xce\xbf' b' ' b'\xce\x93' b'\xcf\x81' b'\xce\xb1'
 b'\xce\xbc' b'\xce\xbc' b'\xce\xb1' b'\xcf\x84' b'\xce\xad' b'\xce\xb1'
 b'\xcf\x82' b' ' b'\xce\xba' b'.' b' ' b'\xce\x92' b'\xce\xbf'
 b'\xcf\x85' b'\xce\xbb' b'\xce\xb3' b'\xce\xb1' b'\xcf\x81' b'\xce\xac'
 b'\xce\xba' b'\xce\xb7' b'\xcf\x82' b' ' b'\xce\xbd' b'\xce\xb1' b' '
 b'\xcf\x83' b'\xcf\x85' b'\xce\xbd' b'\xce\xbf' b'\xce\xb4' b'\xce\xb5'
 b'\xcf\x8d' b'\xcf\x83' b'\xce\xb5' b'\xce\xb9' b' ' b'\xcf\x84'
 b'\xce\xbf' b' ' b'\xce\x9c' b'\xce\xb1' b'\xce\xba' b'\xce\xb1'
 b'\xcf\x81' b'\xce\xb9' b'\xcf\x8c' b'\xcf\x84' b'\xce\xb1' b'\xcf\x84'
 b'\xce\xbf' b' ' b'\xce\x91' b'\xcf\x81' b'\xcf\x87' b'\xce\xb9'
 b'\xce\xb5' b'\xcf\x80' b'\xce\xaf' b'\xcf\x83' b'\xce\xba' b'\xce\xbf'
 b'\xcf\x80' b'\xce\xbf' b' ' b'\xce\x91' b'\xce\xb8' b'\xce\xb7'
 b'\xce\xbd' b'

In [20]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy().decode('utf-8'))

 Παρακαλείται ο Γραμματέας κ. Βουλγαράκης να συνοδεύσει το Μακαριότατο Αρχιεπίσκοπο Αθηνών και πάσης Ελλάδος κ. ΣΕΡΑΦΕΙΜ και τα συνοδεύοντα αυτόν μέλη της Ιεράς Συνόδου κατά την είσοδό τους στην Αίθουσ
α της Βουλής, προκειμένου να τελεσθεί αγιασμός.  . Στη συνέχεια τελείται ο καθιερωμένος αγιασμός.
 Παρακαλείται ο κύριος Γραμματέας να συνοδεύσει την Ιερά Σύνοδο εκτός της Αιθούσης της Βουλής.  . Παρακ
αλείται ο συνάδελφος Βουλευτής κ. Σαδίκ Αμέτ, που ανήκει στο Μωαμεθανικό Θρήσκευμα να προσέλθει και να δώσει τον οριζόμενο από το Σύνταγμα όρκο επί του Κορανίου.  : ~"Ορκίζομαι στο όνομα του Παντοδύναμ
ου Θεού και του μόνου αυτού Προφήτη ο οποίος είναι ο Μωάμεθ να είμαι πιστός στην πατρίδα και το δημοκρατικό πολίτευμα, να υπακούω στο Σύνταγμα και τους νόμους και να εκπληρώνω ευσυνείδητα τα καθήκοντά 
μου".
 Κύριοι συνάδελφοι, παρακαλώ τη Βουλή να εξουσιοδοτήσει το Προεδρείο για την επικύρωση των Πρακτικών της σημερινής συνεδριάσεως.
 Μάλιστα, μάλιστα.
 Η Βουλή παρέσχε τη ζητηθείσα εξουσιοδ

In [21]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in  dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy().decode('utf-8'))
  print("Target:", text_from_ids(target_example).numpy().decode('utf-8'))

Input :  Παρακαλείται ο Γραμματέας κ. Βουλγαράκης να συνοδεύσει το Μακαριότατο Αρχιεπίσκοπο Αθηνών και πάσης Ελλάδος κ. ΣΕΡΑΦΕΙΜ και τα συνοδεύοντα αυτόν μέλη της Ιεράς Συνόδου κατά την είσοδό τους στην Αίθου
Target: Παρακαλείται ο Γραμματέας κ. Βουλγαράκης να συνοδεύσει το Μακαριότατο Αρχιεπίσκοπο Αθηνών και πάσης Ελλάδος κ. ΣΕΡΑΦΕΙΜ και τα συνοδεύοντα αυτόν μέλη της Ιεράς Συνόδου κατά την είσοδό τους στην Αίθουσ


In [24]:
# Batch size
BATCH_SIZE = 32

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((32, 200), (32, 200)), types: (tf.int64, tf.int64)>

In [25]:
# Length of the vocabulary in chars
vocab_size = len(parlvocab)

# The embedding dimension
embedding_dim = 512

# Number of RNN units
rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [27]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [28]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints-parliament'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    verbose=1,
    save_weights_only=True)

The below cells include the results of 10 epochs, 256 embedding dims, 1024 rnn neurons, batch size of 128

In [ ]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
1016/1016 [==============================] - 314s 306ms/step - loss: 2.2163

Epoch 00001: saving model to ./training_checkpoints-parliament/ckpt_1
Epoch 2/10
1016/1016 [==============================] - 312s 306ms/step - loss: 1.0962

Epoch 00002: saving model to ./training_checkpoints-parliament/ckpt_2
Epoch 3/10
1016/1016 [==============================] - 312s 306ms/step - loss: 1.0167

Epoch 00003: saving model to ./training_checkpoints-parliament/ckpt_3
Epoch 4/10
1016/1016 [==============================] - 312s 305ms/step - loss: 0.9799

Epoch 00004: saving model to ./training_checkpoints-parliament/ckpt_4
Epoch 5/10
1016/1016 [==============================] - 313s 306ms/step - loss: 0.9576

Epoch 00005: saving model to ./training_checkpoints-parliament/ckpt_5
Epoch 6/10
1016/1016 [==============================] - 313s 307ms/step - loss: 0.9411

Epoch 00006: saving model to ./training_checkpoints-parliament/ckpt_6
Epoch 7/10
1016/1016 [==============================

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 200, 165) # (batch_size, sequence_length, vocab_size)


In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['πρόεδρε'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

πρόεδρε.
 Συνεπώς, η τροποποίηση του άρθρου 6 θα επιμείνει μόνο η επιχειρηματολογία του κυρίου Υπουργού, όταν ξέρετε ότι ο τραγικός μας χαρακτηρίζει αυτό θα πει ο κ. Αφανά, για το έργο και εν πάση περιπτήσει, αφήστε τώρα; Κύριε Πρόεδρε, παρακαλώ το λέγω για τους Υπουργούς, όσο και γι' αυτό προτείνουμε στον Κοινοβουλευτικό Εκπρόσωπο του Συνασπισμού εγκυματρόποντα και για την αναβολή, ως εξής:«Στο Νομό Αττικής η οικονομία μας λέει ότι η μείωση των τιμών πετρελαίου ή για πολύπλοκτα εγκυβέρνηση υπέρ τους στα θέματα των ενσρεκτικών αρμοδιοτήτων και του οποίου θα αφήσει ο πόλλος χώρος των ή μη ακόμα δεκαντιων στην αναποτελεσματική και ελαστική της λήξη της οικονομίας. Πιστεύουμε ότι ο θεσμό της Κυβέρνησης θα εφαρμόσετε τα ονόματα όσων λήξεων στο Λαό σας καμία επίθετουσαν οι κρίσεις, που μας τα στερήσατε τους καθηγητές, -εδώ απόκτά εκείνα που προ-ΣΕΛΙΔΑ 2395νισμό, ρεβάν, εξάλλου γίνεται για το έργο μας. Και αντιλαμβάνεσθε τι δηλαδή δεν είναι ομολογούμενοι στους υπαλλήλους; Τώρα ας θα επιτρέψο

We will re-train our model with smaller batch size (32), more epochs (30), a larger sequence size (200) and larger embedding dimensions (512) , in order to give it more chances to earn from its mistakes. 
<br> <br> Due to colab limitations, after several attempts, we haven't been able to complete the model (continuous disconnections - prevention from gpu/tpu runtimes. Even after gmail account change & vpn usage)

In [29]:
model_secat = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model_secat.compile(optimizer='adam', loss=loss)
EPOCHS = 30
history = model_secat.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
4066/4066 [==============================] - 807s 198ms/step - loss: 1.5621

Epoch 00001: saving model to ./training_checkpoints-parliament/ckpt_1
Epoch 2/30
4066/4066 [==============================] - 808s 198ms/step - loss: 1.0147

Epoch 00002: saving model to ./training_checkpoints-parliament/ckpt_2
Epoch 3/30
4066/4066 [==============================] - 812s 199ms/step - loss: 0.9828

Epoch 00003: saving model to ./training_checkpoints-parliament/ckpt_3
Epoch 4/30
4066/4066 [==============================] - 806s 198ms/step - loss: 0.9721

Epoch 00004: saving model to ./training_checkpoints-parliament/ckpt_4
Epoch 5/30
4066/4066 [==============================] - 802s 197ms/step - loss: 0.9714

Epoch 00005: saving model to ./training_checkpoints-parliament/ckpt_5
Epoch 6/30
4066/4066 [==============================] - 804s 197ms/step - loss: 0.9786

Epoch 00006: saving model to ./training_checkpoints-parliament/ckpt_6
Epoch 7/30
4066/4066 [==============================

In [ ]:
model_secat.load_weights('./training_checkpoints-parliament/ckpt_10')

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 200, 165) # (batch_size, sequence_length, vocab_size)


In [ ]:
one_step_model_secat = OneStep(model_secat, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['πρόεδρε'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model_secat.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

LSTM attempt (not working)

In [ ]:
model2 = keras.Sequential()
model2.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[BATCH_SIZE, None]))
model2.add(LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(LSTM(256))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))
model2.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam')

In [ ]:
model2_hist = model2.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])